In [3]:
import pandas as pd
import numpy as np
import tensorflow as tensorflow
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adamax
from sklearn.metrics import classification_report,confusion_matrix
import warnings
warnings.filterwarnings("ignore")


In [4]:
train_dr="dataset/Training"
flods=os.listdir(train_dr)
paths=[]
labels=[]
for fold in flods:
    foldpath=os.path.join(train_dr,fold)
    filepaths=os.listdir(foldpath)
    for path in filepaths:
        filepath=os.path.join(foldpath,path)
        paths.append(filepath)
        labels.append(fold)
train_df=pd.DataFrame(data={"File Path":paths,"Labels":labels})
train_df

,File Path,Labels
0,dataset/Training\glioma\Tr-glTr_0000.jpg,glioma
1,dataset/Training\glioma\Tr-glTr_0001.jpg,glioma
2,dataset/Training\glioma\Tr-glTr_0002.jpg,glioma
3,dataset/Training\glioma\Tr-glTr_0003.jpg,glioma
4,dataset/Training\glioma\Tr-glTr_0004.jpg,glioma
...,...,...
5707,dataset/Training\pituitary\Tr-pi_1452.jpg,pituitary
5708,dataset/Training\pituitary\Tr-pi_1453.jpg,pituitary
5709,dataset/Training\pituitary\Tr-pi_1454.jpg,pituitary
5710,dataset/Training\pituitary\Tr-pi_1455.jpg,pituitary


In [5]:
test_dr="dataset/Testing"
flods=os.listdir(test_dr)
paths=[]
labels=[]
for fold in flods:
    foldpath=os.path.join(test_dr,fold)
    filepaths=os.listdir(foldpath)
    for path in filepaths:
        filepath=os.path.join(foldpath,path)
        paths.append(filepath)
        labels.append(fold)
ts_df=pd.DataFrame(data={"File Path":paths,"Labels":labels})
ts_df

,File Path,Labels
0,dataset/Testing\glioma\Te-glTr_0000.jpg,glioma
1,dataset/Testing\glioma\Te-glTr_0001.jpg,glioma
2,dataset/Testing\glioma\Te-glTr_0002.jpg,glioma
3,dataset/Testing\glioma\Te-glTr_0003.jpg,glioma
4,dataset/Testing\glioma\Te-glTr_0004.jpg,glioma
...,...,...
1306,dataset/Testing\pituitary\Te-pi_0295.jpg,pituitary
1307,dataset/Testing\pituitary\Te-pi_0296.jpg,pituitary
1308,dataset/Testing\pituitary\Te-pi_0297.jpg,pituitary
1309,dataset/Testing\pituitary\Te-pi_0298.jpg,pituitary


In [6]:
test_df,vald_df=train_test_split(ts_df,test_size=0.5,random_state=42)
test_df

,File Path,Labels
7,dataset/Testing\glioma\Te-glTr_0007.jpg,glioma
666,dataset/Testing\notumor\Te-no_0060.jpg,notumor
1074,dataset/Testing\pituitary\Te-pi_0063.jpg,pituitary
357,dataset/Testing\meningioma\Te-me_0057.jpg,meningioma
329,dataset/Testing\meningioma\Te-me_0029.jpg,meningioma
...,...,...
1095,dataset/Testing\pituitary\Te-pi_0084.jpg,pituitary
1130,dataset/Testing\pituitary\Te-pi_0119.jpg,pituitary
1294,dataset/Testing\pituitary\Te-pi_0283.jpg,pituitary
860,dataset/Testing\notumor\Te-no_0254.jpg,notumor


In [7]:
gen=ImageDataGenerator()
train_gen=gen.flow_from_dataframe(train_df,x_col="File Path",y_col="Labels",target_size=(224,224),class_mode="categorical",color_mode="rgb",batch_size=16)
test_gen=gen.flow_from_dataframe(test_df,x_col="File Path",y_col="Labels",target_size=(224,224),class_mode="categorical",color_mode="rgb",batch_size=8)
valid_gen=gen.flow_from_dataframe(vald_df,x_col="File Path",y_col="Labels",target_size=(224,224),class_mode="categorical",color_mode="rgb",batch_size=8)

Found 5712 validated image filenames belonging to 4 classes.
Found 655 validated image filenames belonging to 4 classes.
Found 656 validated image filenames belonging to 4 classes.


In [8]:
model=Sequential([
    
    Conv2D(64,kernel_size=(3,3),activation="relu",input_shape=(224,224,3)),
    MaxPooling2D((2,2)),
    Conv2D(32,kernel_size=(3,3),activation="relu"),
    MaxPooling2D((2,2)),
    Conv2D(16,kernel_size=(3,3),activation="relu"),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(64,activation="relu"),
    Dense(32,activation="relu"),
    Dense(4,activation="softmax")
 
])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 32)   │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 52, 52, 16)     │         4,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 26, 26, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 10816)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       692,288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │           132 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 719,380 (2.74 MB)

 Trainable params: 719,380 (2.74 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
model.compile(optimizer=Adamax(learning_rate=0.001),loss="categorical_crossentropy",metrics=["accuracy"])

In [10]:
model.fit(train_gen,validation_data=valid_gen,epochs=25,verbose=1)

Epoch 1/25
357/357 ━━━━━━━━━━━━━━━━━━━━ 90s 246ms/step - accuracy: 0.5297 - loss: 3.7717 - val_accuracy: 0.7759 - val_loss: 0.6262
Epoch 2/25
357/357 ━━━━━━━━━━━━━━━━━━━━ 91s 254ms/step - accuracy: 0.8491 - loss: 0.4205 - val_accuracy: 0.8537 - val_loss: 0.4174
Epoch 3/25
357/357 ━━━━━━━━━━━━━━━━━━━━ 90s 252ms/step - accuracy: 0.9104 - loss: 0.2412 - val_accuracy: 0.7866 - val_loss: 0.5928
Epoch 4/25
357/357 ━━━━━━━━━━━━━━━━━━━━ 89s 250ms/step - accuracy: 0.9343 - loss: 0.1890 - val_accuracy: 0.8948 - val_loss: 0.2779
Epoch 5/25
357/357 ━━━━━━━━━━━━━━━━━━━━ 89s 250ms/step - accuracy: 0.9713 - loss: 0.1014 - val_accuracy: 0.8948 - val_loss: 0.3571
Epoch 6/25
357/357 ━━━━━━━━━━━━━━━━━━━━ 89s 249ms/step - accuracy: 0.9788 - loss: 0.0668 - val_accuracy: 0.9390 - val_loss: 0.2153
Epoch 7/25
357/357 ━━━━━━━━━━━━━━━━━━━━ 88s 246ms/step - accuracy: 0.9870 - loss: 0.0363 - val_accuracy: 0.9146 - val_loss: 0.2833
Epoch 8/25
357/357 ━━━━━━━━━━━━━━━━━━━━ 89s 250ms/step - accuracy: 0.9857 - loss: 0

In [11]:
model.evaluate(train_gen)
model.evaluate(test_gen)
model.evaluate(valid_gen)

357/357 ━━━━━━━━━━━━━━━━━━━━ 34s 94ms/step - accuracy: 1.0000 - loss: 1.1618e-05
82/82 ━━━━━━━━━━━━━━━━━━━━ 5s 66ms/step - accuracy: 0.9539 - loss: 0.3172
82/82 ━━━━━━━━━━━━━━━━━━━━ 4s 45ms/step - accuracy: 0.9472 - loss: 0.2915


[0.3381550908088684, 0.957317054271698]

In [25]:
model.save("model.tflite")

ValueError: Invalid filepath extension for saving. Please add either a `.keras` extension for the native Keras format (recommended) or a `.h5` extension. Use `model.export(filepath)` if you want to export a SavedModel for use with TFLite/TFServing/etc. Received: filepath=model.tflite.

In [27]:
import tensorflow as tf

# Convert the model
converter = tf.lite.TFLiteConverter.from_saved_model("a") # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)